This is the continuous-time dynamics of an IBR like system.

In [1]:
from scipy.integrate import quad
from scipy.stats import beta

In [2]:
def expectation(start, stop):
    if start <=0:
        return quad(lambda x : beta.pdf(x, 1, 2)*x, 0, stop)[0]/quad(lambda x : beta.pdf(x, 1, 2), 0, stop)[0]
    else:
        return quad(lambda x : beta.pdf(x, 1, 2)*x, start, stop)[0]/quad(lambda x : beta.pdf(x, 1, 2), start, stop)[0]

In [3]:
def Y_discrete(Y0, n, b):
    # Unpack the position vector
    s, r1, r2 = Y0
    # Create vectors
    s_list = []
    s_list.append(s)
    r1_list = []
    r1_list.append(r1)
    r2_list = []
    r2_list.append(r2)
    # Iterate
    for i in range(1,n):
        s_move = (.5*(r1_list[i-1] + r2_list[i-1]) - b)/(1-b)
        # Calculate next values
        if s_move < 0:
            s_list.append(0)
            r1_list.append(0)
            r2_list.append(expectation(0,1))
        else:
            s_list.append(s_move)
            r1_list.append(expectation(0, s_list[i]))
            r2_list.append(expectation(s_list[i], 1))
    
                          
    return [s_list, r1_list, r2_list]

In [ ]:
s0 = .999
r20 = expectation(s0,1)
r10 = expectation(0,s0)
s, r1, r2 = Y_discrete([s0, r10, r20], 15, .1666)
plt.plot(range(15), r1, 'o-', color='.5')
plt.plot(range(15), r2, 'o-', color='.5')
plt.plot(range(15), s, 'ko-')
plt.ylim(-.05,1.05)
plt.xlim(-.05,14.05)
plt.ylabel("States and actions", fontsize=15)
plt.xlabel("Iterations", fontsize=15)
plt.savefig('IBR-discrete.png')
plt.show()

In [ ]:
plt.plot(range(15), [1 - beta.cdf(item, 1,2) for item in s], 'o-')
plt.ylim(-.05,1.05)
plt.xlim(-.05,14.05)
plt.ylabel(r'Proportion of $m_2$', fontsize=15)
plt.xlabel("Iterations", fontsize=15)
plt.savefig('IBR-ne-not.png')
plt.show()

In [ ]:
for i in np.linspace(0, .5, num=4):
    s, r1, r2 = Y_discrete([s0, r10, r20], 15, i)
    plt.plot(range(15), [1 - beta.cdf(item, 1,2) for item in s], 'bo-')
    
plt.ylim(-.05,1.05)
plt.xlim(-.05,14.05)
plt.ylabel(r'Proportion of $m_2$', fontsize=15)
plt.xlabel("Iterations", fontsize=15)
plt.savefig('IBR-mult.png')
plt.show()

In [ ]:
def Y_diff(Y0, t, params):
    # Unpack the position vector
    s, r1, r2 = Y0
    b_value = params[0]
    # Calculate next values
    if .5*(r1 + r2) < b_value:
        s_next = 0
        s_diff = -s
        #r1_next = expectation
        r1_diff = -r1
        r2_next = expectation(0,1)
        r2_diff = r2_next - r2
        return [s_diff, r1_diff, r2_diff]
    else:
        s_next = (.5*(r1 + r2) - b_value)/(1-b_value)
        s_diff = s_next - s
        r1_next = expectation(0,s_next)
        r2_next = expectation(s_next, 1)
        r1_diff = r1_next - r1
        r2_diff = r2_next - r2
        return [s_diff, r1_diff, r2_diff]

In [ ]:
Y0 = [.9999, expectation(0,.9999), expectation(.9999, 1)]
t = np.linspace(0, 10, num=50)
Y0_sol = odeint(Y_diff, Y0, t, args=([.3],))

In [ ]:
plt.plot(Y0_sol)
plt.ylim(0,1)
plt.show()

In [ ]:
M_sol = [1 - beta.cdf(item, 1, 2) for item in Y0_sol[:,0]]
plt.plot(M_sol)
plt.ylim(0,1)
plt.show()

In [ ]:
Y0 = [.9999, expectation(0,.9999), expectation(.9999, 1)]
t = np.linspace(0, 10, num=50)
for i in  [n/6.0 for n in range(1,6)]: #np.linspace(.1666,1, num=4):
    Y_sol = odeint(Y_diff, Y0, t, args=([i],))
    M_sol = [1 - beta.cdf(item, 1, 2) for item in Y_sol[:,0]]
    plt.plot(t, M_sol, color='b')
plt.ylim(0,1)
plt.show()

In [ ]:
Y0 = [.9999, expectation(0,.9999), expectation(.9999, 1)]
t = np.linspace(0, 10, num=50)
for i in np.linspace(0, .1666, num=4):
    Y_sol = odeint(Y_diff, Y0, t, args=([i],))
    M_sol = [1 - beta.cdf(item, 1, 2) for item in Y_sol[:,0]]
    plt.plot(t, M_sol, color='k')
plt.ylim(0,1)
plt.show()